<a href="https://colab.research.google.com/github/Prince00954/code-with-prince/blob/main/Live_sketch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# STEP 1: Install required packages
!pip install opencv-python-headless --quiet

# STEP 2: Import required libraries
import cv2
import numpy as np
from IPython.display import display, Javascript
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow

# STEP 3: Function to capture webcam frame
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
      async function takePhoto(quality) {
        const div = document.createElement('div');
        const capture = document.createElement('button');
        capture.textContent = '📷 Capture';
        div.appendChild(capture);
        document.body.appendChild(div);

        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        document.body.appendChild(video);
        video.srcObject = stream;
        await video.play();

        // Resize the output to fit the video
        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        // Wait for Capture to be clicked
        await new Promise((resolve) => capture.onclick = resolve);

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);

        stream.getVideoTracks()[0].stop();
        div.remove();
        video.remove();

        const dataUrl = canvas.toDataURL('image/jpeg', quality);
        return dataUrl;
      }
      ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    import base64
    header, encoded = data.split(',', 1)
    binary = base64.b64decode(encoded)

    with open(filename, 'wb') as f:
        f.write(binary)

    return filename

# STEP 4: Capture photo
image_path = take_photo()

# STEP 5: Apply Live Sketch filter
def sketch_image(img_path):
    img = cv2.imread(img_path)

    if img is None:
        print("Failed to load image!")
        return

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    inv = 255 - gray
    blur = cv2.GaussianBlur(inv, (21, 21), 0)
    sketch = cv2.divide(gray, 255 - blur, scale=256.0)

    cv2_imshow(sketch)
    cv2.imwrite('sketch_output.jpg', sketch)

# STEP 6: Show result
sketch_image(image_path)

<IPython.core.display.Javascript object>